In [32]:
import pandas as pd

d = pd.read_csv("../data/spi_matches.csv", parse_dates=["date"])
d = d[d.league_id == 2411]
d = d[d.score1.isnull()]

dates_to_ignore = [(2019, 2, 22), (2019, 2, 23), (2019, 2, 24), # League Cup Final
                   (2019, 3, 16), (2019, 3, 17), (2019, 3, 18), # FA Cup Quarters
                   (2019, 4,  6), (2019, 4,  7), (2019, 4,  8)] # FA Cup Semis
dates_to_ignore_dt = [pd.datetime(t[0], t[1], t[2]) for t in dates_to_ignore]
# 2019 4 30 is week ten
teams_used = []

In [29]:
# 10 weeks
df = d[(d.date > pd.datetime(2019, 1, 28)) &
       (d.date < pd.datetime(2019, 4, 30)) &
       (~d.date.isin(dates_to_ignore_dt)) &
       (~d.team1.isin(teams_used))]

In [2]:
df.date.unique()

array(['2019-01-29T00:00:00.000000000', '2019-01-30T00:00:00.000000000',
       '2019-02-02T00:00:00.000000000', '2019-02-03T00:00:00.000000000',
       '2019-02-04T00:00:00.000000000', '2019-02-09T00:00:00.000000000',
       '2019-02-10T00:00:00.000000000', '2019-02-11T00:00:00.000000000',
       '2019-02-26T00:00:00.000000000', '2019-02-27T00:00:00.000000000',
       '2019-03-02T00:00:00.000000000', '2019-03-09T00:00:00.000000000',
       '2019-03-30T00:00:00.000000000', '2019-04-13T00:00:00.000000000',
       '2019-04-20T00:00:00.000000000', '2019-04-27T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [3]:
max_per_team = df.groupby(["team1"]).agg("max")["prob1"].reset_index().sort_values("prob1", ascending=False).reset_index(drop=True)
max_per_team

,team1,prob1
0,Liverpool,0.8921
1,Manchester City,0.8485
2,Tottenham Hotspur,0.8031
3,Chelsea,0.7860
4,Arsenal,0.7578
5,Manchester United,0.6768
6,Crystal Palace,0.5783
7,Wolverhampton,0.5626
8,Leicester City,0.5573
9,Southampton,0.5550


In [4]:
max_per_team_a = df.groupby(["team2"]).agg("max")["prob2"].reset_index().sort_values("prob2", ascending=False).reset_index(drop=True)
max_per_team_a.columns = ["team1", "prob2"]

In [5]:
home_away_comparison = pd.merge(max_per_team, max_per_team_a, on=["team1"])
home_away_comparison[home_away_comparison.prob1 < home_away_comparison.prob2]

,team1,prob1,prob2


# No teams have their best match away from home

In [6]:
pd.merge(df, max_per_team.iloc[6:10], how="inner", on=["team1", "prob1"])[["date", "team1", "team2", "prob1"]]

,date,team1,team2,prob1
0,2019-02-09,Southampton,Cardiff City,0.5550
1,2019-03-02,Wolverhampton,Cardiff City,0.5626
2,2019-03-09,Leicester City,Fulham,0.5573
3,2019-03-30,Crystal Palace,Huddersfield Town,0.5783


In [16]:
picks = pd.merge(df, max_per_team.iloc[:10], how="inner", on=["team1", "prob1"])[["date", "team1", "team2", "prob1", "probtie"]]
picks.to_csv("../data/last_man_picks.csv")
picks

,date,team1,team2,prob1,probtie
0,2019-01-29,Arsenal,Cardiff City,0.7578,0.1666
1,2019-01-29,Manchester United,Burnley,0.6768,0.1993
2,2019-02-02,Chelsea,Huddersfield Town,0.7860,0.1728
3,2019-02-09,Southampton,Cardiff City,0.5550,0.2610
4,2019-02-26,Manchester City,West Ham United,0.8485,0.1109
5,2019-03-02,Wolverhampton,Cardiff City,0.5626,0.2653
6,2019-03-09,Leicester City,Fulham,0.5573,0.2554
7,2019-03-30,Crystal Palace,Huddersfield Town,0.5783,0.2760
8,2019-04-13,Tottenham Hotspur,Huddersfield Town,0.8031,0.1545
9,2019-04-27,Liverpool,Huddersfield Town,0.8921,0.0931


In [17]:
picks[picks.probtie == picks.probtie.max()]

,date,team1,team2,prob1,probtie
7,2019-03-30,Crystal Palace,Huddersfield Town,0.5783,0.276


In [20]:
picks["home+draw"] = picks.prob1 + picks.probtie
picks["drawoverhome"] = picks.probtie / picks.prob1
picks

,date,team1,team2,prob1,probtie,home+draw,drawoverhome
0,2019-01-29,Arsenal,Cardiff City,0.7578,0.1666,0.9244,0.219847
1,2019-01-29,Manchester United,Burnley,0.6768,0.1993,0.8761,0.294474
2,2019-02-02,Chelsea,Huddersfield Town,0.7860,0.1728,0.9588,0.219847
3,2019-02-09,Southampton,Cardiff City,0.5550,0.2610,0.8160,0.470270
4,2019-02-26,Manchester City,West Ham United,0.8485,0.1109,0.9594,0.130701
5,2019-03-02,Wolverhampton,Cardiff City,0.5626,0.2653,0.8279,0.471561
6,2019-03-09,Leicester City,Fulham,0.5573,0.2554,0.8127,0.458281
7,2019-03-30,Crystal Palace,Huddersfield Town,0.5783,0.2760,0.8543,0.477261
8,2019-04-13,Tottenham Hotspur,Huddersfield Town,0.8031,0.1545,0.9576,0.192380
9,2019-04-27,Liverpool,Huddersfield Town,0.8921,0.0931,0.9852,0.104360


In [11]:
df[df.date == pd.datetime(2019,4,20)][["team1", "team2", "prob1", "prob2"]]

,team1,team2,prob1,prob2
20241,Arsenal,Crystal Palace,0.5661,0.1945
20242,AFC Bournemouth,Fulham,0.5257,0.2265
20245,Newcastle,Southampton,0.4018,0.3049
20246,Everton,Manchester United,0.3374,0.4037
20248,Cardiff City,Liverpool,0.0828,0.7578
20249,Manchester City,Tottenham Hotspur,0.6104,0.1760
20250,Huddersfield Town,Watford,0.3022,0.4023
20252,Chelsea,Burnley,0.7457,0.0738
20253,Wolverhampton,Brighton and Hove Albion,0.4986,0.2153
20254,West Ham United,Leicester City,0.4150,0.3049


Use United first week and arsenal on the 20th



In [93]:
round(picks.iloc[2:].prob1.product() * .6793 * .5688 * 100, 1)

1.6

In [26]:
# 11 weeks
df = df[(df.date > pd.datetime(2019, 1, 28)) &
        (df.date < pd.datetime(2019, 5, 5)) &
        (~df.date.isin(dates_to_ignore_dt)) &
        (~df.team1.isin(teams_used))]
max_per_team = df.groupby(["team1"]).agg("max")["prob1"].reset_index().sort_values("prob1", ascending=False).reset_index(drop=True)
picks = pd.merge(df, max_per_team.iloc[:11], how="inner", on=["team1", "prob1"])[["date", "team1", "team2", "prob1", "probtie"]]
picks

,date,team1,team2,prob1,probtie
0,2019-01-29,Arsenal,Cardiff City,0.7578,0.1666
1,2019-01-29,Manchester United,Burnley,0.6768,0.1993
2,2019-02-02,Chelsea,Huddersfield Town,0.7860,0.1728
3,2019-02-09,Southampton,Cardiff City,0.5550,0.2610
4,2019-02-26,Manchester City,West Ham United,0.8485,0.1109
5,2019-03-02,Wolverhampton,Cardiff City,0.5626,0.2653
6,2019-03-09,Leicester City,Fulham,0.5573,0.2554
7,2019-03-30,Crystal Palace,Huddersfield Town,0.5783,0.2760
8,2019-04-13,Tottenham Hotspur,Huddersfield Town,0.8031,0.1545
9,2019-04-27,Liverpool,Huddersfield Town,0.8921,0.0931


Everton in week 11

In [33]:
# 12 weeks
df = d[(d.date > pd.datetime(2019, 1, 28)) &
        (~d.date.isin(dates_to_ignore_dt)) &
        (~d.team1.isin(teams_used))]
max_per_team = df.groupby(["team1"]).agg("max")["prob1"].reset_index().sort_values("prob1", ascending=False).reset_index(drop=True)
picks = pd.merge(df, max_per_team.iloc[:12], how="inner", on=["team1", "prob1"])[["date", "team1", "team2", "prob1", "probtie"]]
picks

,date,team1,team2,prob1,probtie
0,2019-01-29,Arsenal,Cardiff City,0.7578,0.1666
1,2019-02-02,Chelsea,Huddersfield Town,0.7860,0.1728
2,2019-02-09,Southampton,Cardiff City,0.5550,0.2610
3,2019-02-26,Manchester City,West Ham United,0.8485,0.1109
4,2019-03-02,Wolverhampton,Cardiff City,0.5626,0.2653
5,2019-03-09,Leicester City,Fulham,0.5573,0.2554
6,2019-03-30,Crystal Palace,Huddersfield Town,0.5783,0.2760
7,2019-04-13,Tottenham Hotspur,Huddersfield Town,0.8031,0.1545
8,2019-04-20,AFC Bournemouth,Fulham,0.5257,0.2478
9,2019-04-27,Liverpool,Huddersfield Town,0.8921,0.0931
